In [1]:
# allreduce_tensor.py
import torch, time
import hivemind

# 1) Start or join the DHT (peer discovery/coordination)
#    First peer: dht = hivemind.DHT(start=True)
#    Other peers: put the printed address from the first peer into initial_peers=["..."]
dht = hivemind.DHT(start = True, initial_peers= ['/ip4/127.0.0.1/tcp/43977/p2p/12D3KooWFgdYqdCCTNwj16syEJpM3BHQpzb56GjK5LssA59SEsTP'])
print("Share this with other peers:", [str(a) for a in dht.get_visible_maddrs()])

Share this with other peers: ['/ip4/127.0.0.1/tcp/36631/p2p/12D3KooWDUjAkNCAvbhqS4mcpG8AE4mgTVhGQ2pkA2k4UhxX8M6M']


In [2]:

# 2) Make a tensor you'd like to all-reduce (sum/average)
local = torch.ones(4) * (torch.randint(1, 10, ()).item())  # e.g., [k, k, k, k]
print("local before:", local.tolist())

# 3) Create an averager for that tensor; all peers must use the SAME prefix
averager = hivemind.averaging.DecentralizedAverager(
    averaged_tensors=[local], dht=dht, start=True, prefix="demo/mei", target_group_size=4, client_mode = False
)

local before: [2.0, 2.0, 2.0, 2.0]


In [6]:
averager.mode

modes = "AveragingMode.CLIENT" if averager.mode == 0 else "AveragingMode.NODE"
modes

'AveragingMode.NODE'

In [24]:
local += torch.ones(4) * (torch.randint(1, 10, ()).item())
print("local before:", local.tolist())

# 4) Run one all-reduce round (blocks until a group forms or times out)
#    By default, it computes the *average* in-place; set averaging_alpha=1 to overwrite with the average.
#    You can pass weight=<float> to do a weighted average.
info = averager.step(timeout=30.0, gather = {'step': 20})
print("group info:", info)

# After step(), `local` now holds the averaged values from all peers in the group.
print("local after:", local.tolist())


local before: [9.0, 9.0, 9.0, 9.0]


KeyboardInterrupt: 

In [ ]:

averager.shutdown()
dht.shutdown()
